## 2018,2019 데이터 합쳐진 데이터프레임으로 온라인 장바구니 만들기

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale, minmax_scale, robust_scale
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# 데이터 시각화 추세 패키지
import seaborn as sns

!pip3 install mlxtend
# 데이터 시각화 패키지
import matplotlib
import matplotlib.pyplot as plt
# 데이터 시각화 추세 패키지
import seaborn as sns
# 한글 패치
matplotlib.rc('font', family='Malgun Gothic')
matplotlib.rc('font', family='Malgun Gothic')
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
# 회귀분석
from statsmodels.formula.api import ols
# qqplot, 회귀분석의 상수항 추가
from statsmodels.api import qqplot, add_constant
# 선형 모델 formula(y~X1+X2+...)
import statsmodels.formula.api as smf
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
# Scale 변환 -> 표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
# 평가함수
# export_graphviz : 나무 구조 생성 및 저장
from sklearn.tree import export_graphviz
# graphviz : 나무 구조 시각화 (.dot 확장자 파일 불러오기 등)
import graphviz

# 데이터 분할 : train, test
from sklearn.model_selection import train_test_split
#예측/회귀 Decision tree
from sklearn.tree import DecisionTreeRegressor
# 예측 / 회귀 Random Forest
from sklearn.ensemble import RandomForestRegressor
#예측/회귀 GradientBoosting
from sklearn.ensemble import GradientBoostingRegressor
# 최적모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# os 환경
import os

from nltk.tokenize import word_tokenize




from statsmodels.tools.eval_measures import rmse
from statsmodels.tools.eval_measures import mse
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true - y_pred))
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [45]:
from nltk.tokenize import word_tokenize

In [3]:
member = pd.read_csv('C:/Users/ParkChihyun/Desktop/3주차 교육자료/빅데이터프로젝트/jeju_member.csv')
member.head()

,customer_id,birth_date,gender,age,agegroup,customer_type,customer_grade,gaip_date,taltoi_date,member_month,gaip_path,flag_member,flag_rcv_sms,flag_rcv_email,flag_rcv_post,zipcode,sigun,dong,ri,address
0,C102666,1976-11-08,남,44,40대,일반,3Gold,2014-10-07,NaN,63,매장,Y,Y,Y,Y,690-170,제주시,연동,NaN,제주 제주시 연동
1,C118113,1975-08-18,여,45,40대,일반,3Gold,2011-12-06,NaN,97,기타,Y,Y,Y,Y,690-122,제주시,아라2동,NaN,제주 제주시 아라2동
2,C118133,1981-03-20,남,39,30대,일반,4Silver,2011-12-07,2014-04-02,28,매장,N,Y,Y,N,699-900,서귀포시,성산읍,NaN,제주 서귀포시 성산읍
3,C118278,1979-06-22,여,41,40대,일반,3Gold,2011-12-11,NaN,97,인터넷,Y,Y,Y,Y,695-791,제주시,애월읍,광령2리,제주 제주시 애월읍 광령2리 제주관광대학
4,C118280,1959-08-28,남,61,60대,일반,3Gold,2011-12-11,2016-05-02,53,기타,N,N,N,N,699-905,서귀포시,성산읍,삼달리,제주 서귀포시 성산읍 삼달리


In [5]:
product = pd.read_csv('C:/Users/ParkChihyun/Desktop/3주차 교육자료/빅데이터프로젝트/jeju_online_merge.csv',  encoding='cp949')
product.head()

,Unnamed: 0,customer_id,purchase_id,store_id,purchase_yearmonth,purchase_weekday,purchase_hour,purchase_date,purchase_time,supply_date,...,item_g_m,item_g_s,item_name,purchase_cnt,unit_cost,purchase_amt,new_purchase_amt,year,month,season
0,0,C121449,C121449_20180307_201,모바일,2018M03,수요일,12,2018-03-07,:0:12:35:44,2018-03-15,...,잎채소,냉이,냉이(200g),1,3150,3150,3150,2018,3,봄
1,1,C121449,C121449_20180307_201,모바일,2018M03,수요일,12,2018-03-07,:0:12:35:44,2018-03-15,...,밀가공,면/기타,당면(500g),1,4800,4800,4800,2018,3,봄
2,2,C121449,C121449_20180307_201,모바일,2018M03,수요일,12,2018-03-07,:0:12:35:44,2018-03-15,...,잎채소,콩나물,항아리콩나물(250g)-제주,1,1300,1300,1300,2018,3,봄
3,3,C121449,C121449_20180307_201,모바일,2018M03,수요일,12,2018-03-07,:0:12:35:44,2018-03-15,...,버섯,느타리버섯,★느타리버섯(300g),1,2500,2500,2500,2018,3,봄
4,4,C121449,C121449_20180307_201,모바일,2018M03,수요일,12,2018-03-07,:0:12:35:44,2018-03-15,...,두부/유부,두부,순두부(400g)참이든-제주,2,1000,2000,2000,2018,3,봄


In [6]:
date_number = set()
for i in product[['purchase_date', 'customer_id']].values:
    date_number.add(tuple(i))    # datenumber라는 세트만들고 그 안에 구매날짜와 id를 튜플형태로 넣는다.

In [7]:
# 빈 데이터프레임 생성
cart = pd.DataFrame(columns=['purchase_date', 'customer_id', '장바구니', 'purchase_cnt', 'purchase_amt', 'store_id']) 


# tqdm(프로그래스바), enumerate(세트에서 한개의 튜플을 차례로 꺼낸다.)
for num, data in tqdm(enumerate(date_number)):
    cart.loc[num, 'purchase_date'] = data[0]
    cart.loc[num, 'customer_id'] = data[1]
    # product data중에서 cart의 값들과 일치하는 값들만 가져와서 temp에 가져온다.
    temp = product.loc[(product['purchase_date'] == data[0]) & (product['customer_id'] == data[1])]
    # temp(같은 날짜, 같은 아이디)의 장바구니 대분류들을 모두 '장바구니_대'칼럼에 저장한다.
    cart.loc[num, '장바구니_대'] = '+'.join(temp['item_g_l'])
    cart.loc[num, '장바구니_중'] = '+'.join(temp['item_g_m'])
    cart.loc[num, '장바구니_소'] = '+'.join(temp['item_g_s'])
    cart.loc[num, '구매수량합계'] = temp['purchase_cnt'].sum()
    cart.loc[num, '구매금액합계'] = temp['purchase_amt'].sum()
    cart.loc[num, '구매매장'] = '+'.join(temp['store_id'])

In [8]:
cart

,purchase_date,customer_id,장바구니,purchase_cnt,purchase_amt,store_id,장바구니_대,장바구니_중,장바구니_소,구매수량합계,구매금액합계,구매매장
0,2018-12-28,C322810,NaN,NaN,NaN,NaN,양념/가루+간식+간식+간식+간식,가루+식사대용+식사대용+식사대용+밀가공,과채가루+식사대용/기타+식사대용/기타+식사대용/기타+만두,6.0,35600.0,인터넷+인터넷+인터넷+인터넷+인터넷
1,2019-03-27,C633120,NaN,NaN,NaN,NaN,채소+채소+간식+간식+반찬+축산물+반찬+채소+축산물+축산물+축산물+채소+채소+간식+...,양념채소+잎채소+식사대용+빵+두부/유부+육가공+어묵/묵+열매채소+유제품+육가공+돼지...,대파+양배추+씨리얼+빵+두부+돼지가공+어묵+기타+우유/산양유+식육추출가공+일반 돼지...,18.0,81250.0,모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바...
2,2019-05-27,C505444,NaN,NaN,NaN,NaN,간식+간식+채소+양념/가루+간식+간식+축산물+축산물+채소+축산물+반찬+채소+채소+채소,밀가공+과자+잎채소+장+과자+과자+육가공+닭+잎채소+닭+두부/유부+말린나물/건재+말...,라면+스낵+쌈채소모음+장류/기타+스낵+스낵+돼지가공+일반 닭+부추+일반 닭+콩가공/...,29.0,64000.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화
3,2019-12-04,C473447,NaN,NaN,NaN,NaN,간식+양념/가루+간식+간식,빵+소스/기타양념+빵+빵,"찐빵,호떡,전통+소스+찐빵,호떡,전통+빵",9.0,29400.0,전화+전화+전화+전화
4,2018-03-05,C129286,NaN,NaN,NaN,NaN,간식+축산물+양념/가루+반찬+양념/가루+채소+주곡+채소+채소+채소,밀가공+꿀+소스/기타양념+두부/유부+기름/식초+잎채소+멥쌀+말린나물/건재+잎채소+양념채소,라면+꿀+기타양념+두부+기름+시금치+백미+숙주나물+쌈채소모음+양파,11.0,74900.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화
...,...,...,...,...,...,...,...,...,...,...,...,...
3005,2018-09-25,C536929,NaN,NaN,NaN,NaN,채소+주곡+양념/가루+반찬+축산물+축산물+채소+채소+간식+간식+양념/가루+간식+축산...,잎채소+멥쌀+소스/기타양념+두부/유부+한우+닭+잎채소+뿌리채소+식사대용+식사대용+가...,상추+백미+기타양념+두부+일반 한우+일반 닭+열무+무+씨리얼+씨리얼+프리믹스+빵+일...,40.0,290000.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전...
3006,2019-11-29,C701813,NaN,NaN,NaN,NaN,축산물+양념/가루,유제품+가루,우유/산양유+프리믹스,23.0,87000.0,전화+전화
3007,2018-11-12,C479966,NaN,NaN,NaN,NaN,간식+채소+간식+간식+채소+채소+채소+채소+반찬,떡+잎채소+빵+떡+양념채소+뿌리채소+잎/줄기채소+뿌리채소+두부/유부,떡+한재미나리+빵+즉석떡+쪽파+무+갓+생도라지+두부,13.0,41000.0,인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷
3008,2019-07-09,C626014,NaN,NaN,NaN,NaN,주곡,멥쌀,백미,2.0,61400.0,전화


In [10]:
cart.to_csv('장바구니online.csv', index=False)

## 카트에 분기 칼럼 추가

In [11]:
df= pd.DataFrame(cart['purchase_date'].str[5:7]) # 월 떼어오기
df=df.apply(pd.to_numeric)
df=df.astype({'purchase_date': int})

df['quarter']='4분기'
df.rename(columns = {'purchase_date' : 'month'}, inplace = True)

for i in range(len(df)):
    if df['month'][i]==1:
        df['quarter'][i]='1분기'
    if df['month'][i]==2:
        df['quarter'][i]='1분기'
    if df['month'][i]==3:
        df['quarter'][i]='1분기'
    if df['month'][i]==4:
        df['quarter'][i]='2분기'
    if df['month'][i]==5:
        df['quarter'][i]='2분기'
    if df['month'][i]==6:
        df['quarter'][i]='2분기'
    if df['month'][i]==7:
        df['quarter'][i]='3분기'
    if df['month'][i]==8:
        df['quarter'][i]='3분기'
    if df['month'][i]==9:
        df['quarter'][i]='3분기'
        
cart['month']=df['month']
cart['quarter']=df['quarter']

cart

<ipython-input-11-7d5919addb4e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='1분기'
<ipython-input-11-7d5919addb4e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='2분기'
<ipython-input-11-7d5919addb4e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='2분기'
<ipython-input-11-7d5919addb4e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,purchase_date,customer_id,장바구니,purchase_cnt,purchase_amt,store_id,장바구니_대,장바구니_중,장바구니_소,구매수량합계,구매금액합계,구매매장,month,quarter
0,2018-12-28,C322810,NaN,NaN,NaN,NaN,양념/가루+간식+간식+간식+간식,가루+식사대용+식사대용+식사대용+밀가공,과채가루+식사대용/기타+식사대용/기타+식사대용/기타+만두,6.0,35600.0,인터넷+인터넷+인터넷+인터넷+인터넷,12,4분기
1,2019-03-27,C633120,NaN,NaN,NaN,NaN,채소+채소+간식+간식+반찬+축산물+반찬+채소+축산물+축산물+축산물+채소+채소+간식+...,양념채소+잎채소+식사대용+빵+두부/유부+육가공+어묵/묵+열매채소+유제품+육가공+돼지...,대파+양배추+씨리얼+빵+두부+돼지가공+어묵+기타+우유/산양유+식육추출가공+일반 돼지...,18.0,81250.0,모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바일+모바...,3,1분기
2,2019-05-27,C505444,NaN,NaN,NaN,NaN,간식+간식+채소+양념/가루+간식+간식+축산물+축산물+채소+축산물+반찬+채소+채소+채소,밀가공+과자+잎채소+장+과자+과자+육가공+닭+잎채소+닭+두부/유부+말린나물/건재+말...,라면+스낵+쌈채소모음+장류/기타+스낵+스낵+돼지가공+일반 닭+부추+일반 닭+콩가공/...,29.0,64000.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화,5,2분기
3,2019-12-04,C473447,NaN,NaN,NaN,NaN,간식+양념/가루+간식+간식,빵+소스/기타양념+빵+빵,"찐빵,호떡,전통+소스+찐빵,호떡,전통+빵",9.0,29400.0,전화+전화+전화+전화,12,4분기
4,2018-03-05,C129286,NaN,NaN,NaN,NaN,간식+축산물+양념/가루+반찬+양념/가루+채소+주곡+채소+채소+채소,밀가공+꿀+소스/기타양념+두부/유부+기름/식초+잎채소+멥쌀+말린나물/건재+잎채소+양념채소,라면+꿀+기타양념+두부+기름+시금치+백미+숙주나물+쌈채소모음+양파,11.0,74900.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화,3,1분기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,2018-09-25,C536929,NaN,NaN,NaN,NaN,채소+주곡+양념/가루+반찬+축산물+축산물+채소+채소+간식+간식+양념/가루+간식+축산...,잎채소+멥쌀+소스/기타양념+두부/유부+한우+닭+잎채소+뿌리채소+식사대용+식사대용+가...,상추+백미+기타양념+두부+일반 한우+일반 닭+열무+무+씨리얼+씨리얼+프리믹스+빵+일...,40.0,290000.0,전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전화+전...,9,3분기
3006,2019-11-29,C701813,NaN,NaN,NaN,NaN,축산물+양념/가루,유제품+가루,우유/산양유+프리믹스,23.0,87000.0,전화+전화,11,4분기
3007,2018-11-12,C479966,NaN,NaN,NaN,NaN,간식+채소+간식+간식+채소+채소+채소+채소+반찬,떡+잎채소+빵+떡+양념채소+뿌리채소+잎/줄기채소+뿌리채소+두부/유부,떡+한재미나리+빵+즉석떡+쪽파+무+갓+생도라지+두부,13.0,41000.0,인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷+인터넷,11,4분기
3008,2019-07-09,C626014,NaN,NaN,NaN,NaN,주곡,멥쌀,백미,2.0,61400.0,전화,7,3분기


## 1분기의 장바구니 분석

In [30]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '1분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.3, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
1,0.517,(두부/유부)
3,0.495,(잎채소)
4,0.324,"(두부/유부, 잎채소)"
0,0.315,(과자)
2,0.311,(밀가공)


In [31]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(잎채소),(두부/유부),0.494768,0.517190,0.324365,0.655589,1.267599,0.068476,1.401844
0,(두부/유부),(잎채소),0.517190,0.494768,0.324365,0.627168,1.267599,0.068476,1.355118


## 2분기의 장바구니 분석

In [25]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '2분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.25, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
2,0.483,(두부/유부)
10,0.439,(잎채소)
4,0.344,(밀가공)
7,0.319,(열매채소)
5,0.302,(빵)
9,0.298,(육가공)
0,0.281,(과일채소)
8,0.271,(유정란)
1,0.269,(과자)
11,0.265,"(두부/유부, 잎채소)"


In [26]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(잎채소),(두부/유부),0.438830,0.482713,0.264628,0.603030,1.249253,0.052799,1.303090
0,(두부/유부),(잎채소),0.482713,0.438830,0.264628,0.548209,1.249253,0.052799,1.242102


## 3분기의 장바구니 분석

In [27]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '3분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.2, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
1,0.442,(두부/유부)
6,0.349,(열매채소)
3,0.344,(밀가공)
9,0.298,(잎채소)
7,0.289,(유정란)
2,0.285,(멥쌀)
8,0.283,(육가공)
4,0.270,(빵)
5,0.223,(양념채소)
10,0.217,"(두부/유부, 열매채소)"


In [28]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(열매채소),(두부/유부),0.348993,0.441611,0.21745,0.623077,1.410919,0.06333,1.481441
0,(두부/유부),(열매채소),0.441611,0.348993,0.21745,0.492401,1.410919,0.06333,1.282522


## 4분기의 장바구니 분석

In [23]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '4분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.2, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
1,0.396,(두부/유부)
10,0.358,(잎채소)
4,0.278,(밀가공)
5,0.274,(빵)
3,0.250,(멥쌀)
6,0.235,(뿌리채소)
9,0.235,(육가공)
7,0.226,(양념채소)
8,0.220,(유정란)
11,0.214,"(두부/유부, 잎채소)"


In [24]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(잎채소),(두부/유부),0.357820,0.395735,0.214455,0.599338,1.514494,0.072853,1.508167
0,(두부/유부),(잎채소),0.395735,0.357820,0.214455,0.541916,1.514494,0.072853,1.401883
